In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 29 kB/s 


In [3]:
!pip install pycaret

     |████████████████████████████████| 264 kB 40.4 MB/s 
     |████████████████████████████████| 1.3 MB 48.5 MB/s 
     |████████████████████████████████| 108 kB 49.5 MB/s 
     |████████████████████████████████| 1.7 MB 49.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 56 kB 5.8 MB/s 
     |████████████████████████████████| 167 kB 69.5 MB/s 
     |████████████████████████████████| 271 kB 77.9 MB/s 
     |████████████████████████████████| 6.8 MB 64.5 MB/s 
     |████████████████████████████████| 14.4 MB 65 kB/s 
     |████████████████████████████████| 248 kB 70.8 MB/s 
     |████████████████████████████████| 2.0 MB 51.0 MB/s 
     |████████████████████████████████| 80 kB 12.7 MB/s 
     |████████████████████████████████| 3.1 MB 55.4 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |███████████████████

In [4]:
!pip install pandas_profiling

In [5]:
!pip install optuna # 하이퍼 파라미터

     |████████████████████████████████| 302 kB 26.9 MB/s 
     |████████████████████████████████| 80 kB 11.0 MB/s 
     |████████████████████████████████| 141 kB 70.4 MB/s 
     |████████████████████████████████| 49 kB 7.2 MB/s 
     |████████████████████████████████| 111 kB 68.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=d8460efbe98452bd6b138851acef3efa69f3a0d58c4da041c817706d9de88933
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [6]:
# for EDA
import pandas_profiling

# for "2. Data Loading"
import pandas as pd

# for "3-1. Feature Generation"
import numpy as np

# for "3-2. Feature Engineering"
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler

# for "4. Modeling with Pycaret"
from pycaret.regression import *

# for "5. Modeling with CatBoostRegressor"
from catboost import CatBoostRegressor
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error



In [7]:
path = '/content/drive/MyDrive/데청캠 프로젝트/dataset/시군구별_인프라수_청년수.csv'
df = pd.read_csv(path, encoding='cp949')
df = df.drop(['Unnamed: 0'], axis=1)
df['청년인구'] = df['청년인구'].str.replace(",","")

In [8]:
df[df['시군구']=='서울_서초구'] 

,시군구,청년인구,병원,사회스포츠,대학교,초중고,공항,소방서,경찰서,소형상업시설(음식점_카페_학원_편의점),대형상업시설,숙박시설,기차역,행정기관,"법원, 교도소",문화시설_영화/공연/미술관박물관,버스터미널
21,서울_서초구,81935,1279,172.0,1,80.0,0,1,13,8169,29,80,0,20.0,6.0,62.0,3.0


In [9]:
df[df['시군구']=='서울_종로구'] 

,시군구,청년인구,병원,사회스포츠,대학교,초중고,공항,소방서,경찰서,소형상업시설(음식점_카페_학원_편의점),대형상업시설,숙박시설,기차역,행정기관,"법원, 교도소",문화시설_영화/공연/미술관박물관,버스터미널
0,서울_종로구,32943,496,28.0,3,60.0,0,1,23,5730,24,201,0,19.0,0.0,367.0,0.0


In [10]:
df[df['시군구']=='서울_중구']

,시군구,청년인구,병원,사회스포츠,대학교,초중고,공항,소방서,경찰서,소형상업시설(음식점_카페_학원_편의점),대형상업시설,숙박시설,기차역,행정기관,"법원, 교도소",문화시설_영화/공연/미술관박물관,버스터미널
1,서울_중구,28277,572,16.0,2,48.0,0,1,17,5383,62,281,1,18.0,0.0,77.0,0.0


In [11]:
df = df.drop([22,0,1])
df.describe()

,병원,사회스포츠,대학교,초중고,공항,소방서,경찰서,소형상업시설(음식점_카페_학원_편의점),대형상업시설,숙박시설,기차역,행정기관,"법원, 교도소",문화시설_영화/공연/미술관박물관,버스터미널
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,464.722222,87.944444,1.694444,90.125000,0.083333,0.930556,8.402778,5008.513889,22.347222,102.888889,2.055556,17.375000,0.638889,20.208333,0.444444
std,257.647386,49.966436,1.733180,41.828603,0.278325,0.454288,2.694071,2042.560020,13.546123,66.372520,2.251065,5.008268,1.154362,14.152029,0.853970
min,28.000000,15.000000,0.000000,14.000000,0.000000,0.000000,3.000000,733.000000,0.000000,17.000000,0.000000,7.000000,0.000000,1.000000,0.000000
25%,259.250000,46.750000,0.000000,59.750000,0.000000,1.000000,6.000000,3584.000000,11.750000,62.000000,0.000000,14.000000,0.000000,10.000000,0.000000
50%,447.000000,76.500000,1.000000,81.000000,0.000000,1.000000,9.000000,4574.000000,21.500000,88.500000,1.500000,17.000000,0.000000,17.000000,0.000000
75%,612.750000,120.000000,3.000000,108.500000,0.000000,1.000000,10.000000,6769.000000,31.000000,118.500000,3.000000,20.250000,1.000000,25.250000,1.000000
max,1279.000000,205.000000,6.000000,192.000000,1.000000,2.000000,15.000000,9410.000000,57.000000,333.000000,9.000000,30.000000,6.000000,77.000000,4.000000


In [12]:
df_set = df.drop(['시군구'], axis=1)
df_set = df_set.astype('float64')
df_set.corr()

# 병원 초중고 경찰서 소형 대형 행정
# 경찰서 법원교도소 병원 소형 숙박

,청년인구,병원,사회스포츠,대학교,초중고,공항,소방서,경찰서,소형상업시설(음식점_카페_학원_편의점),대형상업시설,숙박시설,기차역,행정기관,"법원, 교도소",문화시설_영화/공연/미술관박물관,버스터미널
청년인구,1.000000,0.844645,0.410162,0.231006,0.698321,-0.000482,0.255031,0.598137,0.808659,0.568790,-0.202254,0.147167,0.769622,0.174437,0.217407,0.050909
병원,0.844645,1.000000,0.383949,0.145305,0.584036,-0.103966,0.175640,0.570629,0.849786,0.680556,-0.119261,0.065328,0.760090,0.419847,0.403778,0.161884
사회스포츠,0.410162,0.383949,1.000000,0.385739,0.475724,0.067180,0.148744,0.430196,0.375509,0.213153,-0.022523,-0.080113,0.426427,0.260438,0.155436,-0.093486
대학교,0.231006,0.145305,0.385739,1.000000,0.364029,-0.150854,0.062112,0.198663,0.199903,0.141961,-0.161548,0.036902,0.248663,0.098947,0.278258,0.045465
초중고,0.698321,0.584036,0.475724,0.364029,1.000000,0.114024,0.329558,0.554232,0.780308,0.311235,-0.091246,0.191839,0.705246,0.238678,0.048350,0.225145
공항,-0.000482,-0.103966,0.067180,-0.150854,0.114024,1.000000,0.157807,0.123659,0.058120,-0.142269,0.070652,-0.007493,0.037891,-0.080369,-0.101016,0.079011
소방서,0.255031,0.175640,0.148744,0.062112,0.329558,0.157807,1.000000,0.379925,0.342851,0.045171,-0.062853,0.100236,0.184940,0.246942,0.116201,0.189594
경찰서,0.598137,0.570629,0.430196,0.198663,0.554232,0.123659,0.379925,1.000000,0.583178,0.354650,0.091860,0.189021,0.555468,0.305573,0.249339,0.116997
소형상업시설(음식점_카페_학원_편의점),0.808659,0.849786,0.375509,0.199903,0.780308,0.058120,0.342851,0.583178,1.000000,0.580811,0.033809,0.159906,0.764025,0.428316,0.450174,0.284911
대형상업시설,0.568790,0.680556,0.213153,0.141961,0.311235,-0.142269,0.045171,0.354650,0.580811,1.000000,0.085952,0.082961,0.566685,0.163053,0.328174,-0.054925


## Pandas Profiling

In [ ]:
profile = df_set.profile_report()
profile

Output hidden; open in https://colab.research.google.com to view.

## data preprocessing

In [ ]:
items = df_set.columns
need_list = ['병원', '초중고', '경찰서', '소형상업시설(음식점_카페_학원_편의점)',  '대형상업시설', '행정기관']
del_list = items.drop(need_list)

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

In [ ]:
X = df_set[need_list]
y = df_set['청년인구']

In [ ]:
df_data = pd.concat([y,X] , axis=1)
#X_train.loc[:, scailing_features] = scaler.fit_transform(X_train[scailing_features])
df_data.loc[:,need_list] = std_scaler.fit_transform(df_data[need_list])
df_data

,청년인구,병원,초중고,경찰서,소형상업시설(음식점_카페_학원_편의점),대형상업시설,행정기관
2,52554.0,-0.569557,-1.038228,0.223236,0.023411,0.048527,0.125669
3,67271.0,-0.030182,-0.508581,0.597026,-0.220139,-0.025812,0.326740
4,91126.0,0.364577,-0.340057,1.344607,0.296050,0.271547,-0.075402
5,80217.0,0.517009,-0.340057,1.344607,-0.052513,0.791927,-0.276473
6,83722.0,0.290316,-0.340057,0.223236,-0.295078,-0.620532,0.125669
7,94117.0,0.290316,0.430338,1.344607,0.178712,1.832685,0.728883
8,59943.0,0.012811,-0.821554,0.223236,-0.592860,0.197207,-0.477544
9,61212.0,-0.370223,-0.460432,0.223236,-0.778728,-0.546192,-0.276473
10,101260.0,1.103286,1.658156,0.223236,0.103280,0.197207,0.728883
11,98566.0,0.864867,0.502563,0.970816,0.299501,0.940606,0.125669


In [ ]:
X_scaler = std_scaler.fit_transform(X)
X_scaler

array([[-5.69556903e-01, -1.03822820e+00,  2.23235851e-01,
         2.34114790e-02,  4.85274433e-02,  1.25669402e-01],
       [-3.01823902e-02, -5.08581349e-01,  5.97026113e-01,
        -2.20139116e-01, -2.58124698e-02,  3.26740444e-01],
       [ 3.64577217e-01, -3.40057352e-01,  1.34460664e+00,
         2.96050101e-01,  2.71547182e-01, -7.54016409e-02],
       [ 5.17009144e-01, -3.40057352e-01,  1.34460664e+00,
        -5.25132005e-02,  7.91926574e-01, -2.76472683e-01],
       [ 2.90315508e-01, -3.40057352e-01,  2.23235851e-01,
        -2.95077761e-01, -6.20531774e-01,  1.25669402e-01],
       [ 2.90315508e-01,  4.30338065e-01,  1.34460664e+00,
         1.78711960e-01,  1.83268536e+00,  7.28882529e-01],
       [ 1.28112304e-02, -8.21554487e-01,  2.23235851e-01,
        -5.92860271e-01,  1.97207269e-01, -4.77543726e-01],
       [-3.70222844e-01, -4.60431636e-01,  2.23235851e-01,
        -7.78727830e-01, -5.46191861e-01, -2.76472683e-01],
       [ 1.10328579e+00,  1.65815576e+00,  2.232

In [ ]:
reg = setup(df_data, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = '청년인구', # 목표 변수는 등록 차량 수 이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = False, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
            numeric_features=list(df_data.drop(columns = ['청년인구']).columns), # 모든 변수가 숫자로써의 의미가 있다.
            session_id = 2021,
            fold_shuffle = True
            )

,Description,Value
0,session_id,2021
1,Target,청년인구
2,Original Data,"(72, 7)"
3,Missing Values,False
4,Numeric Features,6
5,Categorical Features,0
6,Transformed Train Set,"(71, 6)"
7,Transformed Test Set,"(1, 6)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [ ]:
top5 = compare_models(n_select = 5, sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,11590.5915,3.094083e+08,16778.5581,0.5992,0.2983,0.2662,0.344
huber,Huber Regressor,11843.6536,3.019978e+08,16217.5979,0.6009,0.3040,0.2775,0.016
catboost,CatBoost Regressor,12089.1123,3.131646e+08,16945.6555,0.6252,0.3387,0.3162,0.445
rf,Random Forest Regressor,12336.3665,3.218884e+08,17219.6715,0.5549,0.3186,0.2895,0.402
ridge,Ridge Regression,12595.9691,3.279132e+08,16968.5252,0.5546,0.3180,0.3001,0.017
llar,Lasso Least Angle Regression,12656.0375,3.381177e+08,17133.6899,0.5308,0.3192,0.3007,0.012
lasso,Lasso Regression,12662.1395,3.385764e+08,17143.1373,0.5299,0.3194,0.3009,0.013
lar,Least Angle Regression,12663.0955,3.386481e+08,17144.6198,0.5298,0.3194,0.3009,0.013
lr,Linear Regression,12663.0996,3.386482e+08,17144.6223,0.5298,0.3194,0.3009,0.259
br,Bayesian Ridge,12686.9170,3.209660e+08,16869.0073,0.5737,0.3204,0.3059,0.012


In [ ]:
models = []
for m in top5:
    models.append(tune_model(m, 
                             optimize = 'MAE', 
                             choose_better = True,
                            n_iter = 10))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,12146.9844,1.702684e+08,13048.6924,0.6547,0.2395,0.2041
1,11674.0400,2.184877e+08,14781.3281,0.7518,0.2533,0.2289
2,17785.1602,3.685239e+08,19196.9766,0.2936,0.2976,0.2909
3,16801.8848,5.280289e+08,22978.8789,-0.2085,0.2184,0.1820
4,12590.9268,2.375657e+08,15413.1660,0.8285,0.3945,0.3740
5,6718.2603,8.012554e+07,8951.2871,0.8790,0.1643,0.1345
6,11301.5488,2.401647e+08,15497.2480,0.5526,0.4356,0.3849
7,7699.3247,8.482778e+07,9210.2002,0.8121,0.1572,0.1499
8,13466.8301,8.051395e+08,28374.9785,0.6435,0.3435,0.2582
9,15532.8496,3.381692e+08,18389.3770,0.8439,0.6844,0.8479


In [ ]:
voting = blend_models(models, optimize = 'MAE')
voting = tune_model(voting, 
                 optimize = 'MAE', 
                 choose_better = True,
                 n_iter = 30)

voting = finalize_model(voting)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,10096.6327,1.469888e+08,12123.8945,0.7020,0.2090,0.1679
1,9877.3838,2.027218e+08,14238.0398,0.7697,0.1931,0.1563
2,12659.0785,2.217664e+08,14891.8221,0.5749,0.2325,0.2083
3,12452.7152,3.089747e+08,17577.6748,0.2928,0.1674,0.1269
4,10038.0057,1.680446e+08,12963.2020,0.8787,0.4262,0.4034
5,5455.2394,7.034229e+07,8387.0314,0.8937,0.1354,0.0939
6,8711.9370,1.741116e+08,13195.1355,0.6757,0.4178,0.3478
7,9977.7368,1.770574e+08,13306.2934,0.6078,0.2063,0.1837
8,12711.2765,7.370637e+08,27148.9172,0.6737,0.5420,0.4827
9,17763.5447,4.484236e+08,21176.0151,0.7929,0.6127,0.7126


In [ ]:
df_data.drop(columns = ['청년인구'])

,병원,초중고,경찰서,소형상업시설(음식점_카페_학원_편의점),대형상업시설,행정기관
2,-0.569557,-1.038228,0.223236,0.023411,0.048527,0.125669
3,-0.030182,-0.508581,0.597026,-0.220139,-0.025812,0.326740
4,0.364577,-0.340057,1.344607,0.296050,0.271547,-0.075402
5,0.517009,-0.340057,1.344607,-0.052513,0.791927,-0.276473
6,0.290316,-0.340057,0.223236,-0.295078,-0.620532,0.125669
...,...,...,...,...,...,...
70,0.599088,0.382188,-0.898135,1.208625,-0.471852,-0.276473
71,-1.136291,-0.725255,-1.271925,-0.937479,-1.438271,-1.482899
72,-1.144108,0.093290,-1.271925,-0.747668,-0.843552,-1.683970
73,-1.011219,0.406263,0.970816,-0.208307,-1.289591,-0.879686


In [ ]:
layer1_pred = voting.predict(df_data.drop(columns = ['청년인구']))
layer1_pred

array([ 52164.42804359,  65303.08259754,  82711.95665497,  79349.18598032,
        74627.97766687,  85866.60446787,  58597.95175798,  54853.13682108,
        96713.2100796 ,  92050.51834804,  63715.38225042,  89241.12886734,
        82191.87333766, 116694.57463363,  81479.56696335,  47572.14091003,
        93254.8883001 ,  79195.99372719, 112793.23334888, 101828.69119957,
       128958.66981905,  94672.52147191,  14510.26065013,  18425.60091263,
        17159.76228423,  17845.29214258,  88629.56875789,  54973.67724885,
        49374.54424651,  50144.98524342,  80216.14482788,  57081.05263775,
        45442.20164222,  25680.5777601 ,  41145.30979617,  36215.43714078,
        37128.41522229,  30427.57654652,  35805.73308343,  69144.22020295,
        34440.52039189,  29443.84314038,  87261.136297  ,  86243.82113013,
       108965.81984867,  46696.65218389,  29496.8261571 ,  11639.18212833,
        77705.02459708,  66562.65109426, 100704.91559738,  96393.72851018,
        52817.34279345,  

In [ ]:
df['청년인구']

2     52554
3     67271
4     91126
5     80217
6     83722
      ...  
70    64670
71    26280
72    41455
73    37605
74    64602
Name: 청년인구, Length: 72, dtype: object

In [ ]:
voting.weights

[0.79, 0.79, 0.99, 0.03, 0.05]